# Data Preprocessing

In [2]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '/Users/orin/Data Science/orins_projects/cloud_data'

train = len(os.listdir(f'{path}/train_images'))
test = len(os.listdir(f'{path}/test_images'))

print(f'There are {train} images in the training set')
print(f'There are {test} images in the testing set')

There are 5546 images in the training set
There are 3698 images in the testing set


In [4]:
train_labels = pd.read_csv('/Users/orin/Data Science/orins_projects/cloud_data/train.csv')

In [5]:
len(train_labels)

22184

In [6]:
train_labels.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


We can see in the labels df above that there is a separate row for each potential label in each image. In other words for example, there are 4 rows for image '0011165.jpg': Fish, Flower, Gravel, and Sugar. 

We will need to split this string and then group each label to one image row and One Hot Encode for present labels.

In [7]:
labels_df = train_labels[~train_labels['EncodedPixels'].isnull()]
labels_df['Image'] = train_labels['Image_Label'].map(lambda x: x.split('_')[0])
labels_df['Class'] = train_labels['Image_Label'].map(lambda x: x.split('_')[1])
classes = labels_df['Class'].unique()
labels_df = labels_df.groupby('Image')['Class'].agg(set).reset_index() ##aggregating classes of each image and reassigning to df with new index

for class_name in classes:      #OHE for loop
    labels_df[class_name] = labels_df['Class'].map(lambda x: 1 if class_name in x else 0)
    
labels_df.head()

,Image,Class,Fish,Flower,Sugar,Gravel
0,0011165.jpg,"{Flower, Fish}",1,1,0,0
1,002be4f.jpg,"{Sugar, Flower, Fish}",1,1,1,0
2,0031ae9.jpg,"{Sugar, Flower, Fish}",1,1,1,0
3,0035239.jpg,"{Flower, Gravel}",0,1,0,1
4,003994e.jpg,"{Sugar, Gravel, Fish}",1,0,1,1


In [8]:
train_labels.loc[train_labels['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()

Sugar     3751
Gravel    2939
Fish      2781
Flower    2365
Name: Image_Label, dtype: int64

Above we see that there are more occurences of some labels than others.

In [ ]:
train_labels.loc[train_labels['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().value_counts()
